In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
X = np.arange(-100, 100, 0.1)
y = np.array([x*9/5+32 for x in X])
print(f"X.shape={X.shape}, y.shape={y.shape}")

X_train = torch.from_numpy(X.reshape(-1,1)).float()
y_train = torch.from_numpy(y.reshape(-1,1)).float()
trainset = torch.utils.data.TensorDataset(X_train, y_train)
valsize = int(len(trainset)*.2)
trainset, valset = torch.utils.data.random_split(trainset, [len(trainset)-valsize, valsize])
print(len(trainset), len(valset))

In [ ]:
BATCH_SIZE = 32

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE)

In [ ]:
xs, ys = next(iter(trainloader))
print(xs.shape, ys.shape)

In [ ]:
model = nn.Sequential(nn.Linear(1, 1)).to(device)
for name, param in model.named_parameters():
    print(name, param)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [ ]:
def train(model, dataloaders:dict, otpimizer, criterion, num_epochs:int, device):
    for epoch in range(1, num_epochs+1):
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            elif phase == "val":
                model.eval()

            with torch.set_grad_enabled(phase == "train"), tqdm(total=len(dataloaders[phase]), unit="batch") as pbar:
                loss_sum = 0
                total = 0
                pbar.set_description(f"Epoch[{epoch}/{num_epochs}]({phase})")
                for batch_idx, (xs, ys) in enumerate(dataloaders[phase]):
                    xs, ys = xs.to(device), ys.to(device)
                    optimizer.zero_grad()
                    output = model(xs)
                    loss = criterion(output, ys)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                    total += xs.size(0)
                    loss_sum += loss.item() * xs.size(0) 

                    running_loss = loss_sum / total

                    pbar.set_postfix({"loss":running_loss})
                    pbar.update(1)

In [ ]:
dataloaders = {"train":trainloader, "val":valloader}
num_epochs = 20

train(model, dataloaders, optimizer, criterion, num_epochs, device)

In [ ]:
for name, param in model.named_parameters():
    print(name, param)